# Temporal Data

In [1]:
import pandas as pd

In [2]:
# Set the stock market (NASDAQ or NYSE)
market = "NASDAQ"
preprocess = False

## Get list of stocks

In [3]:
path = "../Temporal_Relational_Stock_Ranking/data"

file_name = path + "/{}_tickers_qualify_dr-0.98_min-5_smooth.csv".format(market)
stock_csv = pd.read_csv(file_name, header=None)
stock_list = stock_csv.values.squeeze().tolist()
print(len(stock_list))
print(stock_list[:5])

1026
['AABA', 'AAON', 'AAPL', 'AAWW', 'AAXJ']


## Generate DataFrame

In [4]:
# deal with invalid data
def process_data(df):
    df = df.copy()
    # if NASDAQ, remove last row. NASDAQ has 1 extra row
    # with multiple invalid data
    if market == "NASDAQ":
        df = df.drop(df.tail(1).index)
    # if first row is invalid, set its value as the next
    # possible one
    for column in ["open", "high", "low", "close", "volume"]:
        if df.at[0, column] < 0:
            count = 1
            while df.at[count, column] < 0:
                count += 1
            for i in range(count):
                df.at[i, column] = df.at[count, column]
    # iterate through rows setting invalid values as the
    # last temporal value
    for row in df.itertuples():
        if row.Index > 0:
            if row.open < 0:
                df.at[row.Index, "open"] = df.at[row.Index - 1, "open"]
            if row.high < 0:
                df.at[row.Index, "high"] = df.at[row.Index - 1, "high"]
            if row.low < 0:
                df.at[row.Index, "low"] = df.at[row.Index - 1, "low"]
            if row.close < 0:
                df.at[row.Index, "close"] = df.at[row.Index - 1, "close"]
            if row.volume < 0:
                df.at[row.Index, "volume"] = df.at[row.Index - 1, "volume"]
    return df

In [5]:
# generate dataframe function
def generate_df(stock):
    columns = ["day", "open", "high", "low", "close", "volume"]
    df = pd.read_csv("{}/2013-01-01/{}_{}_1.csv".format(path, market, stock), header=None, names=columns)
    df["day"] = df["day"].astype(int)
    if preprocess: # process data
        df = process_data(df) 
    df["tic"] = stock # set tic list
    return df

In [6]:
# First subdataframe
final_df = generate_df(stock_list[0])
final_df

,day,open,high,low,close,volume,tic
0,0,0.270533,0.269522,0.267237,0.263333,0.275333,AABA
1,1,0.271109,0.269741,0.267820,0.263982,0.271219,AABA
2,2,0.271822,0.270097,0.268485,0.264723,0.272316,AABA
3,3,0.271548,0.269697,0.268622,0.265180,0.266009,AABA
4,4,0.270890,0.270122,0.268937,0.265678,0.269574,AABA
...,...,...,...,...,...,...,...
1241,1241,0.951488,0.972700,0.971102,0.961456,0.934183,AABA
1242,1242,0.948882,0.967983,0.969176,0.962736,0.951735,AABA
1243,1243,0.949321,0.964720,0.968531,0.964939,0.962841,AABA
1244,1244,0.952475,0.962416,0.969258,0.967535,0.976964,AABA


In [7]:
# other dataframes
for i in range(1, len(stock_list)):
    df = generate_df(stock_list[i])
    final_df = pd.concat([final_df, df])
final_df = final_df.sort_values(by=["day", "tic"])
final_df

,day,open,high,low,close,volume,tic
0,0,0.270533,0.269522,0.267237,0.263333,0.275333,AABA
0,0,0.238730,0.237522,0.239888,0.240502,0.248031,AAON
0,0,0.424556,0.424768,0.429769,0.442628,0.445018,AAPL
0,0,0.650667,0.648412,0.634492,0.631473,0.667839,AAWW
0,0,0.766565,0.761693,0.759330,0.750061,0.788157,AAXJ
...,...,...,...,...,...,...,...
1245,1245,0.906946,0.917946,0.913736,0.921598,0.911329,ZBRA
1245,1245,0.664203,0.655720,0.639444,0.632873,0.690553,ZEUS
1245,1245,0.974642,0.968477,0.932576,0.916080,0.965211,ZION
1245,1245,0.986084,0.977432,0.963141,0.965654,1.000000,ZIV


## Save DataFrame

In [8]:
final_df.to_csv("../temporal_data/{}_temporal_data.csv".format(market), index=False)